# Least Squares

The basic problem should be familiar to everyone. You have a set of data, and you want to create a model that describes data well enough that it is reasonably predictive. Here's a concrete example that is relevant to anyone working with biological processes: the kinetics of an enzyme catalyzed reaction. We know that the rate of an enzyme-catalyzed reaction will generally depend on the concentration of the reactant, let's call it the substrate. We perform an experiment by varying the concentration of our substrate ($C_S$) and measuring the resultant reaction rate. I've loaded some typical laboratory data into the cell below for visualization; go ahead and plot them using a scatter plot to visualize the relationship between rate and substrate concentration in this system.

In [1]:
CS = [0, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5] #mmol/L
rate = [0, 0.017537467, 0.030941975, 0.080327165, 0.1643835, 0.26569368, 0.745442547, 1.295792328, 2.419014706, 4.0402125, 5.534947297, 5.127217742, 7.074911496]; #mmol/L/min

Looking at the data, I notice two things.  First, the data are not perfect.  This is typical of experimental data!  Second, I can see that, initially, increasing the substrate concentration increases the rate of reaction, but this only occurs up to a certain point. This is referred to as "saturation kinetics" wherein the rate eventually becomes insensitive to additional increases in substrate concentration. This data set looks typical of a Michaelis-Menten mechanism for an enzyme-catalyzed reaction, where the rate is usually described by:

$$rate = \frac{V_{max}C_S}{K_m + C_S}$$

Inspecting the equation, it's easy to see why the reaction rate becomes zero order in $C_S$ at high concentrations.  In this limit, $C_S$ is the dominant term in the denominator, and the rate approaches $V_{max}$, which should give you a hint as to why this parameter is called "$V_{max}$".

It would be useful to my group if I could figure out the values for $V_{max}$ and $K_m$ -- once we know their values, we can predict the rate of that reaction for *any* substrate concentration, which will allow us to scale this reaction and implement it in any system that we need to.  But how do we do that? Looking at the graph, we can see that we have 13 different rate vs. $C_S$ pairs, all of which are noisy and subject to measurement uncertainty.  We didn't do anything wrong *per se*, this is just typical of a laboratory experiment, where there are many sources of error. Our first instinct might be to pick two rate vs. $C_S$ pairs and plug them into the Michaelis-Menten rate equation (I know because 70% of CEN 587 students do this on their exams). That would give us two equations and two unknowns ($V_{max}$ and $K_m$), which we could easily solve using a root finding algorithm. Unfortunately, since none of our measurments are *exactly correct*, we will get different answers for every ($C_S$, rate) pair that we pick. Our system is over-determined, and it is incorrect (and honestly a bit lazy) to try to solve it as a system of equations. Instead, we have to solve it by *minimizing* the error between our measurements and our experiments. Usually, we do this by minimizing some form of the residual sum of squares, which quantifies the level of agreement between measurement values and model predictions. The most basic version is defined based only on the difference between the model and measurement:

$$\textrm{SSE} = \sum_{i = 1}^n\left(\textrm{measurement}_i - \textrm{model}_i\right)^2$$

But this is really just another example of minimizing an objective function! Which we already know how to do! In this case our objective function is the residual sum of squares. Minimizing that objective function by varying model parameters minimizes the residual sum of squares, which is the process of least squares regression. We use this method all the time! Every time you use a trendline in Excel, for example, you are performing a least squares regression (whether you knew it or not).  

Now, let's look at how to formulate and solve this problem in Julia.

**Least Squares Regression of the Michaelis-Menten Model using Experimental Data**

The "saturation kinetics" that we observe in this system suggests a Michaelis-Menten mechanisms, which we model as:

$$rate = \frac{V_{max}C_S}{K_m + C_S}$$

Next, we need to build an objective function that calculates the residual sum of squares between the model prediction and the experimental measurment for our experimental set of substrate concentrations. At the moment, I'm going to use the optimization package we already know, Optim.jl, to solve the least-squares problem.  This will be similar to optimization of multivariate scalar functions in Supplement 10. In this case, our objective function will be a *multivariate scalar function*. It will accept two parameters that we want to find optimum values for -- $K_m$ and $V_{max}$ -- and it will return a single value that we wish to minimize, the residual sum of squares. 

When I construct this objective function, I will need to pass all of the parameters I'm looking to vary in a single array-like structure, so I have to pack $K_m$ and $V_{max}$ into a  collection for this to work. Though it isn't necessary to do so, I often rename them once inside the function scope to make it more readable.  Since I want that function to compute the error between model and measurement, I also need to access my experimental data inside of the function. Best practice is to work with local variables inside of a function (i.e., variables that are only defined inside of the function scope), so I add two extra arguments to my objective function, one representing substrate concentration, and one representing the reaction rate. 

    function temp(par, C, r)
        Vmax  = par[1]
        Km    = par[2]
        model = (Vmax*C)./(Km .+ C)
        SSE = sum((r .- model).^2)
    end

Next, I want to convert that "temp" function into one that just takes var as its only argument, which is required by Optim.jl syntax. But see how I also pass CS and rate to the temp function.  So now, the anonymous function I created has "access" to the values in CS and rate.

    objective = par -> temp(par,CS,rate)

Now that I have an objective function, I can use optim.optimize to minimize it.  We have to provide initial guesses for our parameters as some sort of collection. I'll pass my initial guesses into the minimizer as a 1D array
    
    par0 = [1.,1.]
    solution = optimize(objective, par0)
    solution.minimizer
    
Remember, if you want to use Optim.optimize, you'll need to declare 
    
    using Optim
    
at the top of the cell.

Now, technically, this is an implementation of least squares error minimization, which is generally what we mean when we talk about "nonlinear regression." The family of Optim.jl optimizers are general minimization routines, whereas there are (at least) two specific algorithms that are used for nonlinear least squares. If I'm solving a least squares problem, I generally work with these because they are optimized for least squares problems.  So it is worth considering them here.  They are:

    LsqFit.jl
    
and
    
    LeastSquaresOptim.jl

**LsqFit**
    
LsqFit, is really great for simple model fitting (exactly like what we are doing with the Michaelis-Menten rate law here). 

https://github.com/JuliaNLSolvers/LsqFit.jl

As usual, we have to install it first:

    using Pkg
    Pkg.add("LsqFit")

This one has a pretty unique syntax relative to what we find elsewhere. Instead of writing a function to return your objective, you provide Lsqfit with a function that returns your model prediction only. Lsqfit then uses that model equation to generate the least squares objective function automatically. So that it can do this, you'll also provide Lsqfit with your raw "xdata" and "ydata", in our case, these are substrate concentration and rate respectively.  The basic syntax is straightforward -- you will use the Lsqfit subroutine curve_fit:

    curve_fit(modelfunction, xdata, ydata, parameter_guesses)

Though you can add a number of options as usual. For our specific problem, "xdata" = CS, "ydata" = rate, and "function" will be a function that returns the Michaelis-Menten rate at a specific value of CS, $V_{max}$, and $K_m$). 

When you use Lsqfit, the model function that you create must accept your independent variable as its first argument and a vector of your variable parameters as its second argument.  Any additional parameters will need to be passed using an anonymous function approach as above. We also need to make sure that our model function is *vectorized* with respect to Concentration because Julia will be passing our full set of experimental concentrations into the function as a vector. Code below should get it done:

    using LsqFit
    function modelfun(C, par)
        Vmax = par[1]
        Km   = par[2]
        retval = (Vmax*C)./(Km .+ C)
    end
    
    par0 = [7., 1.]
    solution  = curve_fit(modelfun, CS, rate, par0)
    
One of the things I like about this curve_fit package is that it returns confidence intervals on your parameter estimates from nonlinear regression, which Optim.jl will not give you by default. It will also give you a solution structure with various fields (printed out with fieldnames below)
    
    beta = coef(solution)
    intervals = confidence_interval(solution, 0.05)
    println(beta)
    println(intervals)
    fieldnames(LsqFit.LsqFitResult)


Another very nice feature of curve_fit is that it is really easy to add upper and lower bounds on our parameters.  For example, by including a few keyword arguments (see documentation), I can bound my value of Km between 1 and 10, and my value of Vmax between 0 and 2.

    par0 = [7., 1.]
    lb   = [1., 0.] #Lower bounds on Km and Vmax
    ub   = [10., 2.] #upper bounds on Km and Vmax
    solution  = curve_fit(modelfun, CS, rate, par0, lower = lb, upper = ub)
    solution.param

**LeastSquaresOptim**

Now that we have also covered working with multivariate vector functions (Supplement 11), we can apply those same approaches in using the LeastSquaresOptim package.

https://github.com/matthieugomez/LeastSquaresOptim.jl

Let's go ahead and install that one:

    using Pkg
    Pkg.add("LeastSquaresOptim")

I like this package quite a bit  It it is specifically designed for nonlinear least squares, and it offers some nice options for least squares problems, whereas Optim.jl routines just minimize the function that we supply to them. In general, writing alternative "least squares" algorithms and properly scaling data requires a good understanding of the methods and the data, and the lsqnonlin algorithm is built to handle some of this for us. The other major difference between Optim.optimize and LeastSquaresOptim.optimize (other than the latter being specifically designed for least squares problem) is that Optim.optimize works on a scalar function, while LeastSquaresOptim.optimize works on a vector function.  

Specifically, we provide LeastSquaresOptim.optimize with a function that calculates the vector of residuals, the difference between measurement and model, that includes every data point, and then LeastSquaresOptim.optimize generates the loss function automatically from that full vector of residuals. Revisiting that Michaelis-Menten example, let's load the data again:

In [15]:
CS = [0, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5]; #mmol/L
rate = [0, 0.017537467, 0.030941975, 0.080327165, 0.1643835, 0.26569368, 0.745442547, 1.295792328, 2.419014706, 4.0402125, 5.534947297, 5.127217742, 7.074911496]; #mmol/L/min

Now we'll set up the least squares problem by creating a function that returns a *vector* of residuals for each data point; we still need it to take variable parameters ($V_{max}$ and $K_m$) as a vector input, and we have to pass that input as the first argument to the function.  We also need to pass our experimental concentrations and rates in order to calculate model predictions and residuals:

    function resid(par, Cexp, rexp)
        Vmax     = par[1]
        Km       = par[2]
        rmod     = (Vmax*Cexp)./(Km .+ Cexp)
        retval   = (rexp .- rmod)
    end

Now that we have a function that returns residuals when we provide 1) the parameter set, 2) experimental concentrations, and 3) experimental rates, we are ready to use lsqnonlin to find optimal values for $V_{max}$ and $K_m$.  I'm going to create an anonymous function so that I can pass CS and rate to an objective function that takes only the parameters as arguments, and then I'll use LeastSquaresOptim.optimize on that objective function. 

LeastSquaresOptim.optimize is a bit different than Optim.optimize, which we used in Supplement 10.  It has the syntax:

    optimize(functionname, parameterguess, methodname)
    
Where methodname is either Dogleg() or LevenbergMarquardt().  You can also dial in a lot of non-default options, but we'll save that for if/when we actually benefit from doing so.

    residuals = par -> resid(par, CS, rate) 
    par0 = [7, 0.4]
    optimize(residuals, par0, Dogleg())

Sometimes, you may find that one package or another gives the best performance or otherwise includes features that you need.  Usually, I try each of the above on least squares problems and decide which of them best fit for whatever I'm solving at the moment.